In [36]:
import automatization
from automatization.ple_statistics import PleAuto
import pandas as pd
import os
import time
import json
from importlib import reload
import numpy as np
from automatization.ple_statistics import PleAuto
from matplotlib import pyplot as plt
reload(automatization.ple_statistics)
from automatization.ple_statistics import PleAuto
from scipy.constants import speed_of_light

In [37]:
pa = PleAuto(
    ple_gui, 
    ple_optimize_logic, 
    laser_scanner_logic,
    poi_manager_logic, 
    scanning_optimize_logic, 
    spectrometer, 
    spectrometerlogic,
    cobolt,
    ibeam_smart
)

In [ ]:
# check scanning range scaling
high_finesse_wavemeter_remote.start_acquisition()
high_finesse_wavemeter_remote._wavelength_buffer = []
wave_buff = high_finesse_wavemeter_remote.get_wavelength_buffer()
factor_sigma = float((np.max(np.array(wave_buff)) - np.min(np.array(wave_buff)) )/ 0.02)
factor_sigma

KeyboardInterrupt: 

In [ ]:
w1 = high_finesse_wavemeter_remote.get_current_wavelength()

In [ ]:
w2 = high_finesse_wavemeter_remote.get_current_wavelength()

In [ ]:
(w2 - w1) * 1000 

-0.0029040780304967484

In [38]:
factor_sigma = 0.509

In [ ]:
cobolt.enable_modulated()
cobolt.set_laser_modulated_power(power = 5)
ibeam_smart.disable()

In [ ]:
pa.one_pulse_repump("blue")

In [39]:
results = []
power_steps = 3 * np.logspace(1.5, 2, 10, endpoint=True).astype(int)[::-1]
folder = r"C:\Users\yy3\Documents\data\SnV\cooldown_06_03_2023_95NA\197\2R_F6"
os.mkdir(folder:=os.path.join(folder, r"auto_B"))

for poi_name in poi_manager_logic.poi_names:
    if poi_name == "ref":
        continue
    results_poi = {}
    os.mkdir(folder_defect := os.path.join(folder, poi_name))
    results_poi["poi_name"] = poi_name
    results_poi["factor_sigma"] = factor_sigma
    ibeam_smart.disable()
    # go to a defect
    pa.set_resonant_power(power = 300)
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 5)
    #ibeam_smart.enable()
    switchlogic.set_state("ScanningMode", 'Wavemeter')
    scanner_gui._osd.change_settings({'scan_frequency': {"x": 8, "y": 8, "z": 10},
                                    "scan_resolution": {"x": 10, "y": 10, "z":80},
                                    "scan_range": {"x": 2e-6, "y": 2e-6, "z": 4e-6}})
    scanner_gui._osd.accept()
    #configure slow scanning for the wavemeter scanning optimizations

    pa.go_to_poi(poi_name, opt_times=1)


    switchlogic.set_state("ScanningMode", 'NI')
    time.sleep(0.2)

    res = pa.do_ple_scan(lines = 2)
    pa.save_ple(tag = "full_range",
            poi_name=poi_name, folder_name=folder_defect)
    results_poi.update({
        "ple_scans" : {
        "full_range" : ple_data_logic.last_saved_files_paths 
            }
        })


    scan_range_constr = laser_scanner_logic.scanner_constraints.axes["a"].value_range

    if res["center"].stderr > 1e3 or res["sigma"].stderr > res["sigma"].value or (res["center"].value - 2 * res["sigma"].value < scan_range_constr[0]) or (res["center"].value + 2 * res["sigma"].value > scan_range_constr[1]): 
        print("NO PLE or it is at the rim")
        pa.save_ple(tag = "failed_PLE_scan",
            poi_name=poi_name, folder_name=folder_defect)
        raise Exception("No PLE")
        #continue
    #since there is a PLE -- optimize_all
    pa.go_to_ple_target(res["center"].value)
    #get the center wavelength:
    pa.optimize_ple()
    wavelength = high_finesse_wavemeter_remote.get_current_wavelength()
    results_poi["wavelength, THz"] = float(wavelength)

    time.sleep(0.5)

    scanner_gui._osd.change_settings({'scan_frequency': {"x": 50, "y": 50, "z": 50},
                                        "scan_resolution": {"x": 15, "y": 15, "z":80},
                                        "scan_range": {"x": 1e-6, "y": 1e-6, "z": 4e-6}})
    scanner_gui._osd.accept()
    pa.optimize_all()

    results_poi["xyz"] = tuple(float(coord) for coord in poi_manager_logic.poi_anchors[poi_name])
    #Now we now where the defect is!
    res = pa.do_ple_scan(lines = 1)
    fine_scan_range = ( 
        res["center"].value - 5 * res["sigma"].value ,
        res["center"].value + 5 * res["sigma"].value
                    )
    pa.set_resonant_power(power = power_steps[1])

    res = pa.do_ple_scan(lines = 2, in_range=fine_scan_range)

    pa.save_ple(tag = "5sigma_range",
                poi_name=poi_name, folder_name=folder_defect)
    time.sleep(0.5)

    results_poi.update({
        "ple_scans" : {
        "5sigma_range" : ple_data_logic.last_saved_files_paths 
        }
    })


    # now we run saturation with a cw repump
    os.mkdir(saturation_folder := os.path.join(folder_defect, "saturation"))
    results_poi["saturation"] = {}
    idx_no_ple = None
    ibeam_smart.disable()
    for idx, power in enumerate(power_steps):
        
        #405 repump
        cobolt.enable_modulated()
        cobolt.set_laser_modulated_power(power = 3)
        pa.set_resonant_power(power = power_steps[1])
        fine_scan_range = ( 
        res["center"].value - 6 * res["sigma"].value ,
        res["center"].value + 6 * res["sigma"].value
                    )
        res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range)
        pa.set_resonant_power(power = int(power))
        fine_scan_range = ( 
        res["center"].value - 3 * res["sigma"].value ,
        res["center"].value + 3 * res["sigma"].value
                    )
        res = pa.do_ple_scan(lines = 2, in_range=fine_scan_range)
        
        #save_results
        pa.save_ple(tag = f"{power}",
            poi_name=poi_name, folder_name=saturation_folder)
        #save_plots
        if res["center"].stderr is None or res["sigma"].stderr is None or res["sigma"].value is None:
            idx_no_ple = idx
            break
        if res["center"].stderr > 1e3 or res["sigma"].stderr * 4 > res["sigma"].value or res["amplitude"].stderr * 3 > res["amplitude"].value: 
            #ple is gone.
            idx_no_ple = idx
            break

        
        #get the center wavelength:
        
        pa.go_to_ple_target(res["center"].value)
        pa.optimize_ple()
        wavelength = high_finesse_wavemeter_remote.get_current_wavelength()

        # #charge_lifetime:
        # for i in range(10):
        #     pa.optimize_ple()
        #     time.sleep(5)

        results_poi.update({"saturation": 
                            {f"{power}":
                                {"scan_data": ple_data_logic.last_saved_files_paths,
                                "sigma": res["sigma"].value,
                                "sigma_stderr": res["sigma"].stderr,
                                "wavelength": wavelength
                                }
                                }})
    if idx_no_ple is None:
        continue
        
    # photostability study
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 5)
    pa.set_resonant_power(power = int(power_steps[0]))
    time.sleep(0.5)
    res = pa.do_ple_scan(lines = 1)

    pa.set_resonant_power(power = int(power_steps[idx_no_ple - 3]))
    time.sleep(1)
    #ple_gui._osd.change_settings({"scan_range": {"a": 5000}}) # 3GHz
    #ple_gui._osd.accept()

    pa.set_resonant_power(power = int(power_steps[idx_no_ple - 1])) #take the last but one visible ple power
    #reionize with blue
    fine_scan_range = ( 
    res["center"].value - 5 * res["sigma"].value ,
    res["center"].value + 5 * res["sigma"].value
                )
    res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range)

    os.mkdir(photostability := os.path.join(folder_defect, "photostability_405"))


    cobolt.set_laser_modulated_power(power = 20)
    cobolt.disable_modulated()
    ibeam_smart.disable()

    #walk off resonance:
    pa.go_to_ple_target(0)


    pa.one_pulse_repump("blue")

    fine_scan_range = ( 
    res["center"].value - 5 * res["sigma"].value ,
    res["center"].value + 5 * res["sigma"].value
                )
    res = pa.do_ple_scan(lines = 10, in_range=fine_scan_range)
    #cobolt.set_laser_modulated_power(power = 10)
    pa.save_ple(tag = "photostability", folder_name=photostability) 

    results_poi.update({"photostability_405": {
                        "power": power,
                        "scan_data": ple_data_logic.last_saved_files_paths,
                        "sigma": res["sigma"].value,
                        "sigma_stderr": res["sigma"].stderr
                        }}
                        )
    cobolt.set_laser_modulated_power(power = 5)



    # take spectrum to estimate SOC
    pa.set_resonant_power(power = 0)
    time.sleep(1)
    ibeam_smart.enable()
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 100)
    pa.set_resonant_power(power = 0)
    time.sleep(1)
    pa.take_spectrum()

    pa.save_spectrum(name_tag=f"{poi_name}_blueNgreen", folder_path=folder_defect)
    results_poi["spectrum_data"] = spectrometerlogic.last_saved_path

    spectrometer._mw.data_widget.fit_widget.sigDoFit.emit("DoubleLorentzian")
    time.sleep(0.2)
    # spectrometer.fit_results.params["center_1"].value
    results_poi["SOC, GHz"] = float(speed_of_light / spectrometer.fit_results.params["center_2"].value - speed_of_light / spectrometer.fit_results.params["center_1"].value)

    ibeam_smart.disable()

    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 5)

    with open(os.path.join(folder_defect, f'results_{poi_name}.txt'), 'w') as file:
        file.write(json.dumps(results_poi))

EOFError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [ ]:
pulsestreamer._seq = pulsestreamer.pulse_streamer.createSequence()
pulse_pattern_1 = [(100, 0), (200, 1), (80, 0), (300, 1), (60, 0)]
pulsestreamer._seq.setDigital(6, pulse_pattern_1)
pulsestreamer.pulser_on()
pulsestreamer.pulser_off()
_hist_bin_width = 10 * 1000 # ns * ps
_hist_record_length = 10
hist_channel = 1
pulsestreamer._seq = pulsestreamer.pulse_streamer.createSequence()
pulse_pattern_5 = [(_hist_record_length, 0), (10, 1)]
pulsestreamer._seq.setDigital(5, pulse_pattern_5)
timetagger.sigToggleHist.emit({'hist': (_hist_bin_width, _hist_record_length, int(hist_channel), False)})

In [ ]:
# #charge_lifetime:

for i in range(10):
    pa.optimize_ple()
    time.sleep(5)

In [ ]:
ibeam_smart.disable()

cobolt.enable_modulated()
cobolt.set_laser_modulated_power(power = 5)

In [34]:
cobolt.enable_modulated()

In [ ]:
#reionize with green
os.mkdir(photostability := os.path.join(folder_defect, "photostability_520"))
for i in range(3):
    #walk off resonance:
    pa.go_to_ple_target(0)
    
    pa.one_pulse_repump("green")
    
    res = pa.do_ple_scan(lines = 30, in_range=fine_scan_range)
    #cobolt.set_laser_modulated_power(power = 10)
    pa.save_ple(tag = "photostability", folder_name=photostability) 

results_poi.update({"photostability_520": {
                    "power": power,
                    "scan_data": ple_data_logic.last_saved_files_paths,
                    "sigma": res["sigma"].value,
                    "sigma_stderr": res["sigma"].stderr
                    }}
                    )

In [ ]:

pa.set_resonant_power(power = int(power_steps[idx_no_ple - 1])) #take the last but one visible ple power
#reionize with blue
os.mkdir(photostability := os.path.join(folder_defect, "photostability_405"))

cobolt.disable_modulated()
ibeam_smart.disable()

for i in range(3):
    #walk off resonance:
    pa.go_to_ple_target(0)
    
    pa.one_pulse_repump("blue")
    res = pa.do_ple_scan(lines = 10, in_range=fine_scan_range)
    #cobolt.set_laser_modulated_power(power = 10)
    pa.save_ple(tag = "photostability", folder_name=photostability) 

results_poi.update({"photostability_405": {
                    "power": power,
                    "scan_data": ple_data_logic.last_saved_files_paths,
                    "sigma": res["sigma"].value,
                    "sigma_stderr": res["sigma"].stderr
                    }}
                    )
#reionize with green
os.mkdir(photostability := os.path.join(folder_defect, "photostability_520"))
for i in range(3):
    #walk off resonance:
    pa.go_to_ple_target(0)
    
    pa.one_pulse_repump("green")
    
    res = pa.do_ple_scan(lines = 30, in_range=fine_scan_range)
    #cobolt.set_laser_modulated_power(power = 10)
    pa.save_ple(tag = "photostability", folder_name=photostability) 

results_poi.update({"photostability_520": {
                    "power": power,
                    "scan_data": ple_data_logic.last_saved_files_paths,
                    "sigma": res["sigma"].value,
                    "sigma_stderr": res["sigma"].stderr
                    }}
                    )

# take spectrum to estimate SOC
pa.set_resonant_power(power = 0)
time.sleep(1)
ibeam_smart.enable()
cobolt.enable_modulated()
cobolt.set_laser_modulated_power(power = 100)
pa.set_resonant_power(power = 0)
time.sleep(1)
pa.take_spectrum()

pa.save_spectrum(name_tag=f"{poi_name}_blueNgreen", folder_path=folder_defect)
results_poi["spectrum_data"] = spectrometerlogic.last_saved_path

spectrometer._mw.data_widget.fit_widget.sigDoFit.emit("DoubleLorentzian")
time.sleep(0.2)
# spectrometer.fit_results.params["center_1"].value
results_poi["SOC"] = speed_of_light / spectrometer.fit_results.params["center_2"].value - speed_of_light / spectrometer.fit_results.params["center_1"].value



with open(os.path.join(folder_defect, f'results_{poi_name}.txt'), 'w') as file:
    file.write(json.dumps(results_poi))

In [ ]:
power_steps = 3 * np.logspace(0.4, 2, 15, endpoint=True)
folder = r"C:\Users\yy3\Documents\data\SnV\cooldown_06_03_2023_95NA\197\2R_F6"
os.mkdir(folder:=os.path.join(folder, r"test"))

In [ ]:
poi_name = "def2"

In [ ]:
results_poi = {}
results_poi["ple_scans"] = {}
os.mkdir(folder_defect := os.path.join(folder, poi_name))

# go to a defect
results_poi["poi_name"] = poi_name
pa.set_resonant_power(power = 300)
cobolt.enable_modulated()
cobolt.set_laser_modulated_power(power = 5)
#ibeam_smart.enable()
switchlogic.set_state("ScanningMode", 'Wavemeter')
scanner_gui._osd.change_settings({'scan_frequency': {"x": 5, "y": 5, "z": 5},
                                    "scan_resolution": {"x": 10, "y": 10, "z":60},
                                    "scan_range": {"x": 2e-6, "y": 2e-6, "z": 4e-6}})
#configure slow scanning for the wavemeter scanning optimizations

pa.go_to_poi(poi_name)


switchlogic.set_state("ScanningMode", 'NI')
time.sleep(0.2)

res = pa.do_ple_scan(lines = 2)



scan_range_constr = laser_scanner_logic.scanner_constraints.axes["a"].value_range
if res["center"].stderr > 1e3 or res["sigma"].stderr > res["sigma"].value or (res["center"].value - 2 * res["sigma"].value < scan_range_constr[0]) or (res["center"].value + 2 * res["sigma"].value > scan_range_constr[1]): 
    print("NO PLE or it is at the rim")

#since there is a PLE -- optimize_all
pa.go_to_ple_target(res["center"].value)
#get the center wavelength:
pa.optimize_ple()
wavelength = high_finesse_wavemeter_remote.get_current_wavelength()
results_poi["wavelength, THz"] = float(wavelength)
time.sleep(0.2)
pa.save_ple(tag = "full_range",
            poi_name=poi_name, folder_name=folder_defect)
results_poi["ple_scans"]["full_range"] = ple_data_logic.last_saved_files_paths 
time.sleep(0.5)

scanner_gui._osd.change_settings({'scan_frequency': {"x": 50, "y": 50, "z": 50},
                                    "scan_resolution": {"x": 30, "y": 30, "z":80},
                                    "scan_range": {"x": 1.2e-6, "y": 1.2e-6, "z": 4e-6}})
pa.optimize_all()

results_poi["xyz"] = tuple(float(coord) for coord in poi_manager_logic.poi_anchors[poi_name])
#Now we now where the defect is!

fine_scan_range = ( 
    res["center"].value - 5 * res["sigma"].value ,
    res["center"].value + 5 * res["sigma"].value
                )
pa.set_resonant_power(power = 270)

res = pa.do_ple_scan(lines = 5, in_range=fine_scan_range)

pa.save_ple(tag = "5sigma_range",
            poi_name=poi_name, folder_name=folder_defect)
time.sleep(0.5)
results_poi["ple_scans"]["5sigma_range"] = ple_data_logic.last_saved_files_paths 


# take spectrum to estimate SOC
pa.set_resonant_power(power = 0)

ibeam_smart.enable()
cobolt.enable_modulated()
cobolt.set_laser_modulated_power(power = 100)

pa.take_spectrum()

pa.save_spectrum(name_tag=f"{poi_name}_blueNgreen", folder_path=folder_defect)
results_poi["spectrum_data"] = spectrometerlogic.last_saved_path

spectrometer._mw.data_widget.fit_widget.sigDoFit.emit("DoubleLorentzian")
time.sleep(0.2)
spectrometer.fit_results.params["center_1"].value
results_poi["SOC"] = 299792458 / spectrometer.fit_results.params["center_2"].value - 299792458 / spectrometer.fit_results.params["center_1"].value



SyntaxError: 'continue' not properly in loop (2599500502.py, line 25)

In [ ]:

# turn on the CW repump
ibeam_smart.disable()
cobolt.enable_modulated()
cobolt.set_laser_modulated_power(power = 10)
#perform one ple scan to determine if there is a PLE
res = pa.do_ple_scan(lines = 1, in_range = None)
if res["center"].stderr > 1e3 or res["sigma"].stderr > res["sigma"].value: 
    continue
#since there is a PLE -- optimize_all
pa.go_to_ple_target(res["center"].value)
#get the center wavelength:
wavelength = high_finesse_wavemeter_remote.get_current_wavelength()
results_poi["wavelength, THz"] = float(wavelength)
time.sleep(0.5)

pa.optimize_all()

#do a san again to determine a finer range to scan
res = pa.do_ple_scan(lines = 5)
#it makes sense to save here
pa.save_ple(tag = "full_range",
            poi_name=poi_name, folder_name=folder_defect)

fine_scan_range = ( 
    res["center"].value - 5 * res["sigma"].value ,
    res["center"].value + 5 * res["sigma"].value
                )
pa.set_resonant_power(power = 270)

res = pa.do_ple_scan(lines = 5, in_range=fine_scan_range)

pa.save_ple(tag = "5sigma_range",
            poi_name=poi_name, folder_name=folder_defect)

fine_scan_range = ( 
    res["center"].value - 4 * res["sigma"].value ,
    res["center"].value + 4 * res["sigma"].value
                )

res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range)
# here we have a somewhat fine range for the scan. 
#Let's perform a saturation scan with a blue cw repump ON
os.mkdir(saturation_folder := os.path.join(folder_defect, "saturation"))

idx_no_ple = None

for idx, power in enumerate(power_steps):
    pa.go_to_ple_target(res["center"].value)
    pa.set_resonant_power(power = power)
    pa.optimize_ple()
    res = pa.do_ple_scan(lines = 10, in_range=fine_scan_range)
    #save_results
    pa.save_ple(tag = f"{power}",
        poi_name=poi_name, folder_name=saturation_folder)
    #save_plots
    
    if res["center"].stderr > 1e3 or res["sigma"].stderr * 3 > res["sigma"].value: 
        #ple is gone.
        idx_no_ple = idx
        break
    time.sleep(1)
    results_poi.update({"saturation": 
                        {f"{power}":
                            {"scan_data": ple_data_logic.last_saved_files_paths,
                            "sigma": str(res["sigma"].value) + " ± " str(res["sigma"].stderr)
                            }
                            }})

if idx_no_ple is None:
    continue
# photostability study

pa.set_resonant_power(power = power_steps[idx_no_ple - 2]) #take the last visible ple power
#reionize with blue
os.mkdir(photostability := os.path.join(folder_defect, "photostability_405"))


for i in range(3):
    #walk off resonance:
    pa.go_to_ple_target(0)
    time.sleep(0.5)
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 20)
    time.sleep(2)
    cobolt.disable_modulated()
    ibeam_smart.disable()
    res = pa.do_ple_scan(lines = 50, in_range=fine_scan_range)
    #cobolt.set_laser_modulated_power(power = 10)
    pa.save_ple(tag = "photostability", folder_name=photostability) 

#reionize with green
os.mkdir(photostability := os.path.join(folder_defect, "photostability_520"))
for i in range(3):
    #walk off resonance:
    pa.go_to_ple_target(0)
    time.sleep(0.5)
    ibeam_smart.enable()
    time.sleep(2)
    cobolt.disable_modulated()
    ibeam_smart.disable()
    
    res = pa.do_ple_scan(lines = 20, in_range=fine_scan_range)
    #cobolt.set_laser_modulated_power(power = 10)
    save_ple(tag = "photostability", folder_name=photostability) 

with open(os.path.join(folder_defect, f'results_{poi_name}.txt'), 'w') as file:
    file.write(json.dumps(results_poi)) # use `json.loads` to do the reverse
#with open(os.path.join(folder_defect, f'results_{poi_name}.txt')) as file:
#results = json.load(file) # use `json.loads` to do the reverse

In [ ]:
power_steps = 3 * np.logspace(0.4, 2, 15, endpoint=True)
folder = r"C:\Users\yy3\Documents\data\SnV\cooldown_06_03_2023_95NA\197\2R\poiA"
os.mkdir(folder:=os.path.join(folder, r"automated_measurements"))
results = []
for poi_name in poi_manager_logic.poi_names:
    if poi_name == "ref":
        continue
    results_poi = {}
    os.mkdir(folder_defect := os.path.join(folder, poi_name))
    
    # go to a defect
    results_poi["poi_name"] = poi_name
    results_poi["xyz"] = tuple(float(coord) for coord in poi_manager_logic.poi_anchors[poi_name])
    #ibeam_smart.enable()
    switchlogic.set_state("ScanningMode", 'Wavemeter')
    
    pa.go_to_poi(poi_name)

    #set max resonant power
    pa.set_resonant_power(power = 300)
    # turn on the CW repump
    ibeam_smart.disable()
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power = 10)
    #perform one ple scan to determine if there is a PLE
    res = pa.do_ple_scan(lines = 1, in_range = None)
    if res["center"].stderr > 1e3 or res["sigma"].stderr > res["sigma"].value: 
        continue
    #since there is a PLE -- optimize_all
    pa.go_to_ple_target(res["center"].value)
    #get the center wavelength:
    wavelength = high_finesse_wavemeter_remote.get_current_wavelength()
    results_poi["wavelength, THz"] = float(wavelength)
    time.sleep(0.5)

    pa.optimize_all()

    #do a san again to determine a finer range to scan
    res = pa.do_ple_scan(lines = 5)
    #it makes sense to save here
    pa.save_ple(tag = "full_range",
             poi_name=poi_name, folder_name=folder_defect)

    fine_scan_range = ( 
        res["center"].value - 5 * res["sigma"].value ,
        res["center"].value + 5 * res["sigma"].value
                    )
    pa.set_resonant_power(power = 270)

    res = pa.do_ple_scan(lines = 5, in_range=fine_scan_range)

    pa.save_ple(tag = "5sigma_range",
             poi_name=poi_name, folder_name=folder_defect)

    fine_scan_range = ( 
        res["center"].value - 4 * res["sigma"].value ,
        res["center"].value + 4 * res["sigma"].value
                    )
    
    res = pa.do_ple_scan(lines = 1, in_range=fine_scan_range)
    # here we have a somewhat fine range for the scan. 
    #Let's perform a saturation scan with a blue cw repump ON
    os.mkdir(saturation_folder := os.path.join(folder_defect, "saturation"))

    idx_no_ple = None

    for idx, power in enumerate(power_steps):
        pa.go_to_ple_target(res["center"].value)
        pa.set_resonant_power(power = power)
        pa.optimize_ple()
        res = pa.do_ple_scan(lines = 10, in_range=fine_scan_range)
        #save_results
        pa.save_ple(tag = f"{power}",
            poi_name=poi_name, folder_name=saturation_folder)
        #save_plots
        
        if res["center"].stderr > 1e3 or res["sigma"].stderr * 3 > res["sigma"].value: 
            #ple is gone.
            idx_no_ple = idx
            break
        time.sleep(1)
        results_poi.update({"saturation": 
                            {f"{power}":
                             {"scan_data": ple_data_logic.last_saved_files_paths,
                              "sigma": str(res["sigma"].value) + " ± " str(res["sigma"].stderr)
                              }
                             }})

    if idx_no_ple is None:
        continue
    # photostability study

    pa.set_resonant_power(power = power_steps[idx_no_ple - 2]) #take the last visible ple power
    #reionize with blue
    os.mkdir(photostability := os.path.join(folder_defect, "photostability_405"))

  
    for i in range(3):
        #walk off resonance:
        pa.go_to_ple_target(0)
        time.sleep(0.5)
        cobolt.enable_modulated()
        cobolt.set_laser_modulated_power(power = 20)
        time.sleep(2)
        cobolt.disable_modulated()
        ibeam_smart.disable()
        res = pa.do_ple_scan(lines = 50, in_range=fine_scan_range)
        #cobolt.set_laser_modulated_power(power = 10)
        pa.save_ple(tag = "photostability", folder_name=photostability) 

    #reionize with green
    os.mkdir(photostability := os.path.join(folder_defect, "photostability_520"))
    for i in range(3):
        #walk off resonance:
        pa.go_to_ple_target(0)
        time.sleep(0.5)
        ibeam_smart.enable()
        time.sleep(2)
        cobolt.disable_modulated()
        ibeam_smart.disable()
        
        res = pa.do_ple_scan(lines = 20, in_range=fine_scan_range)
        #cobolt.set_laser_modulated_power(power = 10)
        save_ple(tag = "photostability", folder_name=photostability) 
    
    with open(os.path.join(folder_defect, f'results_{poi_name}.txt'), 'w') as file:
        file.write(json.dumps(results_poi)) # use `json.loads` to do the reverse
    #with open(os.path.join(folder_defect, f'results_{poi_name}.txt')) as file:
    #results = json.load(file) # use `json.loads` to do the reverse

all_results = pd.DataFrame(results)
all_results.to_csv("ple_statistics" , sep=',')